In [25]:
# Importing required libraries
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Conv2D, MaxPool2D, Flatten
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [26]:
# Defining the training path of the images and the .csv file for Labels and Value
train = pd.read_csv('annotations.csv')
#test = pd.read_excel(r'C:\Users\HP\OneDrive - iitgn.ac.in\Desktop\IITD hack\SoML-50\test.xlsx')
train_folder = 'data'
#test_folder = r'C:\Users\HP\OneDrive - iitgn.ac.in\Desktop\IITD hack\SoML-50\test_imgs'

In [27]:
train.head()

,Image,Label,Value
0,1.jpg,infix,7
1,2.jpg,postfix,0
2,3.jpg,infix,21
3,4.jpg,infix,5
4,5.jpg,prefix,12


In [28]:
train['Value'] = train['Value'].astype('str')

In [29]:
train_gen = ImageDataGenerator(rescale=1./255)
train_data = train_gen.flow_from_dataframe(dataframe=train, directory=train_folder, x_col='Image',
                                           y_col='Label', batch_size=64, shuffle=False,
                                           class_mode='categorical', target_size=(64, 192))

Found 50000 validated image filenames belonging to 3 classes.


In [30]:
image_h, image_w = 64, 192

In [31]:
# Creating the Layers of our model with tensorflow's Sequential API
model = Sequential()

# convolutional layers
model.add(Conv2D(50, kernel_size=(3, 3), strides=(1, 1), activation='relu', input_shape=(64, 192, 3)))

model.add(Conv2D(32, kernel_size=(3, 3), strides=(1, 1), activation='relu'))
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, kernel_size=(3, 3), strides=(1, 1), activation='relu'))
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(128, kernel_size=(3, 3), strides=(1, 1), activation='relu'))
model.add(MaxPool2D(pool_size=(2, 2)))

model.add(Conv2D(256, kernel_size=(3, 3), strides=(1, 1), activation='relu'))
model.add(MaxPool2D(pool_size=(2, 2)))

# flatten output of conv
model.add(Flatten())

# hidden layer
model.add(Dense(500, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(250, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(10, activation='relu'))
model.add(Dropout(0.2))

# output layer
model.add(Dense(3, activation='softmax'))

In [32]:
# compiling the sequential model
model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer='adam')



In [34]:
tf.__version__


'2.8.0'

In [22]:
# training the model for 2 epochs
history = model.fit(train_data, epochs=2, steps_per_epoch=780)

Epoch 1/2
780/780 [==============================] - 40s 50ms/step - loss: 0.2021 - accuracy: 0.8944
Epoch 2/2
780/780 [==============================] - 38s 49ms/step - loss: 0.0191 - accuracy: 0.9947


In [13]:
# import pickle
# pickle.dump(model, open('model.pkl', 'wb'))
model.save('model')

INFO:tensorflow:Assets written to: model/assets


In [19]:
# saving the model as an h5 file to be used later in inference1.py and inference2.py
model.save('Task1Model.h5')

## Lets do the task 2 but for doing this task we shall take the Sequeeze and Exicitation Network 

In [10]:
def squeeze_excitation_layer(input_layer, out_dim, ratio, conv):
  squeeze = tf.keras.layers.GlobalAveragePooling2D()(input_layer)
  
  excitation = tf.keras.layers.Dense(units=out_dim / ratio, activation='relu')(squeeze)
  excitation = tf.keras.layers.Dense(out_dim,activation='sigmoid')(excitation)
  excitation = tf.reshape(excitation, [-1,1,1,out_dim])
  
  scale = tf.keras.layers.multiply([input_layer, excitation])
  
  if conv:
    shortcut = tf.keras.layers.Conv2D(out_dim,kernel_size=1,strides=1,
                                      padding='same',kernel_initializer='he_normal')(input_layer)
    shortcut = tf.keras.layers.BatchNormalization()(shortcut)
  else:
    shortcut = input_layer
  out = tf.keras.layers.add([shortcut, scale])
  return out


In [13]:
def conv_block(input_layer,filters):
  layer = tf.keras.layers.Conv2D(filters,kernel_size=1,strides=1,
                                 padding='same',kernel_initializer='he_normal',
                                 use_bias=False)(input_layer)
  layer = tf.keras.layers.BatchNormalization()(layer)
  layer = tf.keras.layers.ReLU()(layer)

  layer = tf.keras.layers.Conv2D(filters,kernel_size=3,strides=1,
                                 padding='same',kernel_initializer='he_normal',
                                 use_bias=False)(layer)
  layer = tf.keras.layers.BatchNormalization()(layer)
  layer = tf.keras.layers.ReLU()(layer)

  layer = tf.keras.layers.Conv2D(filters*4,kernel_size=1,strides=1,
                                 padding='same',kernel_initializer='he_normal',
                                 use_bias=False)(layer)
  layer = tf.keras.layers.BatchNormalization()(layer)
  layer = tf.keras.layers.ReLU()(layer)
  return layer

def SE_ResNet50(input_w,input_h,include_top):
  model_input = tf.keras.layers.Input(shape=(input_w,input_h,3))
  identity_blocks =[3, 4, 6, 3]
  #Block 1
  layer = tf.keras.layers.Conv2D(64,kernel_size=3,strides=1,
                                 padding='same',kernel_initializer='he_normal',
                                 use_bias=False)(model_input)
  layer = tf.keras.layers.BatchNormalization()(layer)
  layer = tf.keras.layers.ReLU()(layer)
  block_1 =  tf.keras.layers.MaxPooling2D(3, strides=2, padding='same')(layer)
  #Block 2
  block_2 = conv_block(block_1,64)
  block_2 = squeeze_excitation_layer(block_2, out_dim=256, ratio=32.0, conv=True)
  for _ in range(identity_blocks[0]-1):
    block_2 = conv_block(block_1,64)
    block_2 = squeeze_excitation_layer(block_2, out_dim=256, ratio=32.0, conv=False)

  #Block 3
  block_3 = conv_block(block_2,128)
  block_3 = squeeze_excitation_layer(block_3, out_dim=512, ratio=32.0, conv=True)
  for _ in range(identity_blocks[1]-1):
    block_3 = conv_block(block_2, 128)
    block_3 = squeeze_excitation_layer(block_3, out_dim=512, ratio=32.0, conv=False)


  #Block 4
  block_4 = conv_block(block_3,256)
  block_4 = squeeze_excitation_layer(block_4, out_dim=1024, ratio=32.0, conv=True)
  for _ in range(identity_blocks[2]-1):
    block_4 = conv_block(block_3, 256)
    block_4 = squeeze_excitation_layer(block_4, out_dim=1024, ratio=32.0, conv=False)


  #Block 5
  block_5 = conv_block(block_4,512)
  block_5 = squeeze_excitation_layer(block_5, out_dim=2048, ratio=32.0, conv=True)
  for _ in range(identity_blocks[2]-1):
    block_5 = conv_block(block_4, 512)
    block_5 = squeeze_excitation_layer(block_5, out_dim=2048, ratio=32.0, conv=False)
    
    
  if include_top:
    pooling = tf.keras.layers.GlobalAveragePooling2D()(block_5)
    model_output = tf.keras.layers.Dense(3,
                                        activation='softmax')(pooling)
  
    model = tf.keras.models.Model(model_input,model_output)
  else:
    model = tf.keras.models.Model(model_input,block_5)
  model.summary()
  return model

model = SE_ResNet50(image_w,image_h,include_top=True)


Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 192, 64, 3)  0           []                               
                                ]                                                                 
                                                                                                  
 conv2d_67 (Conv2D)             (None, 192, 64, 64)  1728        ['input_2[0][0]']                
                                                                                                  
 batch_normalization_62 (BatchN  (None, 192, 64, 64)  256        ['conv2d_67[0][0]']              
 ormalization)                                                                                    
                                                                                            

 multiply_25 (Multiply)         (None, 96, 32, 512)  0           ['re_lu_79[0][0]',               
                                                                  'tf.reshape_25[0][0]']          
                                                                                                  
 add_25 (Add)                   (None, 96, 32, 512)  0           ['re_lu_79[0][0]',               
                                                                  'multiply_25[0][0]']            
                                                                                                  
 conv2d_107 (Conv2D)            (None, 96, 32, 256)  131072      ['add_25[0][0]']                 
                                                                                                  
 batch_normalization_102 (Batch  (None, 96, 32, 256)  1024       ['conv2d_107[0][0]']             
 Normalization)                                                                                   
          

                                                                                                  
 global_average_pooling2d_39 (G  (None, 2048)        0           ['add_37[0][0]']                 
 lobalAveragePooling2D)                                                                           
                                                                                                  
 dense_81 (Dense)               (None, 3)            6147        ['global_average_pooling2d_39[0][
                                                                 0]']                             
                                                                                                  
Total params: 5,601,595
Trainable params: 5,589,947
Non-trainable params: 11,648
__________________________________________________________________________________________________


In [14]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 192, 64, 3)  0           []                               
                                ]                                                                 
                                                                                                  
 conv2d_67 (Conv2D)             (None, 192, 64, 64)  1728        ['input_2[0][0]']                
                                                                                                  
 batch_normalization_62 (BatchN  (None, 192, 64, 64)  256        ['conv2d_67[0][0]']              
 ormalization)                                                                                    
                                                                                            

 multiply_25 (Multiply)         (None, 96, 32, 512)  0           ['re_lu_79[0][0]',               
                                                                  'tf.reshape_25[0][0]']          
                                                                                                  
 add_25 (Add)                   (None, 96, 32, 512)  0           ['re_lu_79[0][0]',               
                                                                  'multiply_25[0][0]']            
                                                                                                  
 conv2d_107 (Conv2D)            (None, 96, 32, 256)  131072      ['add_25[0][0]']                 
                                                                                                  
 batch_normalization_102 (Batch  (None, 96, 32, 256)  1024       ['conv2d_107[0][0]']             
 Normalization)                                                                                   
          

                                                                                                  
 global_average_pooling2d_39 (G  (None, 2048)        0           ['add_37[0][0]']                 
 lobalAveragePooling2D)                                                                           
                                                                                                  
 dense_81 (Dense)               (None, 3)            6147        ['global_average_pooling2d_39[0][
                                                                 0]']                             
                                                                                                  
Total params: 5,601,595
Trainable params: 5,589,947
Non-trainable params: 11,648
__________________________________________________________________________________________________


In [15]:
starter_learning_rate = 1e-2
end_learning_rate = 1e-5
decay_steps = 80000
learning_rate_fn = tf.keras.optimizers.schedules.PolynomialDecay(starter_learning_rate, decay_steps, end_learning_rate, 
                                                                 power=0.8)

model.compile(loss=tf.losses.SparseCategoricalCrossentropy
              (from_logits=True), 
              optimizer=tf.keras.optimizers.SGD(learning_rate=learning_rate_fn), 
              metrics=[tf.metrics.SparseCategoricalAccuracy()])


callbacks = [tf.keras.callbacks.ModelCheckpoint('SE_model.h5', 
                                       save_best_only=True, 
                                       save_weights_only=True,
                                       monitor='loss')]

In [19]:
model.fit(train_data, epochs=10,steps_per_epoch=780, callbacks=callbacks)

Epoch 1/10


/home/dwip.dalal/anaconda3/envs/pal/lib/python3.8/site-packages/tensorflow/python/util/dispatch.py:1082: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)
2022-04-06 05:02:08.419398: W tensorflow/core/common_runtime/bfc_allocator.cc:275] Allocator (GPU_0_bfc) ran out of memory trying to allocate 3.39GiB with freed_by_count=0. The caller indicates that this is not a failure, but may mean that there could be performance gains if more memory were available.
2022-04-06 05:02:08.419475: W tensorflow/core/common_runtime/bfc_allocator.cc:275] Allocator (GPU_0_bfc) ran out of memory trying to allocate 3.39GiB with freed_by_count=0. The caller indicates that this is not a failure, but may mean that there could be performance gains if more memory were available.
2022-04-06 05:02:08.674825: W tensorfl

InvalidArgumentError: Graph execution error:

Detected at node 'Squeeze' defined at (most recent call last):
    File "/home/dwip.dalal/anaconda3/envs/pal/lib/python3.8/runpy.py", line 194, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "/home/dwip.dalal/anaconda3/envs/pal/lib/python3.8/runpy.py", line 87, in _run_code
      exec(code, run_globals)
    File "/home/dwip.dalal/anaconda3/envs/pal/lib/python3.8/site-packages/ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "/home/dwip.dalal/.local/lib/python3.8/site-packages/traitlets/config/application.py", line 846, in launch_instance
      app.start()
    File "/home/dwip.dalal/anaconda3/envs/pal/lib/python3.8/site-packages/ipykernel/kernelapp.py", line 707, in start
      self.io_loop.start()
    File "/home/dwip.dalal/.local/lib/python3.8/site-packages/tornado/platform/asyncio.py", line 199, in start
      self.asyncio_loop.run_forever()
    File "/home/dwip.dalal/